![Washu Math](https://sites.wustl.edu/scao/files/2020/10/Screen-Shot-2020-10-25-at-1.03.49-PM.png)


# Math Circle Fall 2020 Introduction

Welcome to a math circle session like you have never seen before! Today we will learn how computers learn to tell cats from dogs using machine learning! and some Python language.

![](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)

If you have already registered on Kaggle, now please click the **COPY and EDIT** button on the upper right corner.
![](https://sites.wustl.edu/scao/files/2020/10/Screen-Shot-2020-10-25-at-1.09.19-PM.png)

This a markdown cell. We can write words in this cell. 

## Headers
Hello

### smaller header

In [ ]:
print("This is a code cell.")
print("Hello world")
print(f"Hello world, time is {3} o'clock")

In [ ]:
# Simple variable assignment
# This is a comment
# x variable like 3x = 5
x = 5/3

In [ ]:
y = x*3
print(y)

In [ ]:
# simple calculations
# ** means exponential
print(2**5)
print(2**3*3) # computer 2**3 first, 8 is multiplied with 3
print(1/2+1) # 1/2 first, +1 computed secondly

In [ ]:
print(2***5) # unfortunately tetration is not built-in Python

# Logic
Computer are really good at computing (recognizing numbers), following **EXACT and CLEAR** instructions. However, computer is not so good at many things (used to). For examples, recognizing cats and dogs from photos. 

![](https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg)

In [ ]:
# browse the Cats vs dog competition

In [ ]:
# == vs =
a = 2 # we let a be 2

In [ ]:
type(a)

In [ ]:
dir(a)

In [ ]:
# == is checking whether a variable is equal to whatever on the right side of ==
a == 3

In [ ]:
s = 'David'
print(type(s))

In [ ]:
False is not True

In [ ]:
a = 5.
type(a)

In [ ]:
a = 7
type(a)

In [ ]:
s1 = [a, s]
print(s1)

In [ ]:
type({2,3})

In [ ]:
dict1 = {'Shuhao': 'instructor'}

In [ ]:
# simple logical checking == or using "is"
dict1['Shuhao'] == 'student'

In [ ]:
1 is 0

In [ ]:
# simple if-then condition
# flow control

if 1 is 1:
    print("1 is 1")

if 1 is 0:
    print("1 is 0")

In [ ]:
# range(a,b) the integer greater than or equal to a but less than b
for i in range(0,5):
    if i > 2: # press Tab to indent
        print(i)

# Computer can learn from examples!

Just like us! Imagine we are preparing an exam, teachers will give us some practice exams (with answers available), we will train ourselves by doing these practice problems, honing our skills, then in the actual exam, we will be able to tackle exam problems without knowing the answer beforehand (hopefully). The examples are call data.

First let us load some packages into our system.

In [ ]:
import os
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
dir(pandas)

In [ ]:
dir(pd)

In [ ]:
import site
site.getsitepackages()

In [ ]:
help(pd)

In [ ]:
print(os.listdir("../input/dogs-vs-cats/"))

Let us unzip the compressed images in the `train.zip` and `test1.zip` (this may take a while).

In [ ]:
!unzip -q '../input/dogs-vs-cats/train.zip'
!unzip -q '../input/dogs-vs-cats/test1.zip'

## Prepare the data

In [ ]:
filenames = os.listdir("./train")
print(filenames[:10])

In a computer system, we need to represent "cat" or "dog" these abstract words into 0s and 1s so that computer can understand! We store our data in a Dataframe (like the inventory list in Minecraft).

In [ ]:
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(0)
    else:
        categories.append(1)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
print(df.head(20))

Now let us view a sample image (randomly chosen).

In [ ]:
len(filenames)

In [ ]:
from keras.preprocessing.image import load_img

In [ ]:
sample = random.choice(filenames)
image = load_img("./train/"+sample)
fig = plt.figure()
fig.set_size_inches(6,6)
plt.imshow(image);

# Deep learning model

In [ ]:
from keras import layers, applications, optimizers, callbacks
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Model, load_model
from keras.utils import plot_model, to_categorical

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 6
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.3
x = Dropout(0.3)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

model.summary()

In [ ]:
plot_model(model, to_file='/model_vgg16.png', show_shapes=True)

# Preprocessing the data for the model

In [ ]:
df['category'] = df['category'].astype('str')

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.1)
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

# validate_df = validate_df.sample(n=100).reset_index() # use for fast testing code purpose
# train_df = train_df.sample(n=1800).reset_index() # use for fast testing code purpose

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=16,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    class_mode='binary',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train/",  
    x_col='filename',
    y_col='category',
    class_mode='binary',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

## Augment the data

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
#     class_mode='binary'
)
plt.figure(figsize=(12, 12))
for i in range(0, 9):
    plt.subplot(3, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

# Test if our model can recognize this image!

In [ ]:
test_filenames = os.listdir("./test1/")
test_df = pd.DataFrame({
    'filename': test_filenames[:128]
})

nb_samples = test_df.shape[0]
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    batch_size=batch_size,
    target_size=(image_size, image_size),
    shuffle=False
)

In [ ]:
# this may take a while
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
threshold = 0.5
test_df['category'] = np.where(predict > threshold, 1,0)

## check prediction results

Without training, the model just assigns every images it sees as a "dog" ("0" label).

In [ ]:
sample_test = test_df.sample(n=9).reset_index()
sample_test.head()
plt.figure(figsize=(12, 12))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./test1/"+filename, target_size=(256, 256))
    plt.subplot(3, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')')
plt.tight_layout()
plt.show()

# Now let's load a trained model

Imaging we have spent three days doing practice exam for the actual exam!

In [ ]:
model = load_model('../input/vgg16catsvsdogs/model_0_vgg16.h5')
model.summary()

## Let this model see the images again and check the results

In [ ]:
# this may take a while
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
threshold = 0.5
test_df['category'] = np.where(predict > threshold, 1,0)

In [ ]:
sample_test = test_df.sample(n=9).reset_index()
sample_test.head()
plt.figure(figsize=(12, 12))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./test1/"+filename, target_size=(256, 256))
    plt.subplot(3, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')')
plt.tight_layout()
plt.show()

## Pretty accurate! isn't it

# How this computer algorithm achieves that?!
long story...first we have to learn how computer represent images.

![](https://sites.wustl.edu/scao/files/2020/10/linear_dogs.jpg)

### Computer stores image as a "matrix"

In [ ]:
# example of scalar, vector, matrix
s = 1.34992

In [ ]:
v = [1, 3, 5.2]
print(v)

In [ ]:
m = [[1,3], [8,9]]
print(m)

In [ ]:
m = np.array(m)
print(type(m))
print(m)

In [ ]:
# example of imshow
a = np.array([[0,4], [2,10]])
plt.imshow(a);

## Let us load an image from Pokemon dataset

In [ ]:
pokemon_filename = os.listdir("../input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg/")

In [ ]:
random_pokemon = random.choice(pokemon_filename)
G = plt.imread("../input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg/"+random_pokemon)
plt.imshow(G)

## But what is G????

In [ ]:
G

In [ ]:
m.shape

In [ ]:
G.shape # 3 represents the Red, Green, Blue color 

In [ ]:
# indexing
G1 = G[:,:,0] # take the red channel

In [ ]:
G1.shape

# G is a tensor!

In [ ]:
# show only 1 color channel
plt.imshow(G1)

## computer stores these images as tensors!

The name of the software package we are using is called tensorflow!
![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1200px-TensorFlowLogo.svg.png)

# How computer learns?

Let us load some pokemon data.

In [ ]:
pkmn = pd.read_csv('../input/pokemon/Pokemon.csv')

By just looking at a Pokemon's attributes (features), we will know which pokemon it is without knowing its name.

In [ ]:
pkmn.sample(20)

In [ ]:
sns.countplot(pkmn['Generation']) # sns means seaborn

The Generation and Legendary variables were added to the dataset.
We are going to add a step here to drop the variables so that the presentation is simpler.

In [ ]:
pkmn = pkmn.drop(['Generation', 'Legendary'],1)

## Features visualization

In [ ]:
sns.jointplot(x="HP", y="Speed", data=pkmn);

In [ ]:
sns.jointplot(x="Attack", y="Defense", data=pkmn, kind='hex');

## Now let us go back to the VGG16 model!

# Summary:

1. Computer represents data as vector, matrix, or tensor.
2. Computer learns how to classify images using its features.
3. Our Neural network model (VGG16) uses mathematical operations to extract features from images.

Email: s.cao@wustl.edu